In [ ]:
import os
import pandas as pd
import numpy as np
import dalmatian as dm
import signal
import ipdb
import sys
sys.path.append('../../..')
from JKBio import GCPFunction as gcp
from JKBio import Helper as h
import re
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secrets.json', '~/.storage.json')
%load_ext autoreload
%autoreload 2

In [ ]:
folder = 'gs://ccle_bams/'

In [ ]:
samples = os.popen('gsutil -m ls -al '+folder +'**.bam').read().split('\n')

In [ ]:
sizes = {'gs://' + val.split('gs://')[1].split('#')[0]: int(val.split(" 2019-")[0] if "2019-" in val else val.split(" 2020-")[0]) for val in samples[:-2]}

In [ ]:
sizes

In [ ]:
# can we find more wes for these ??
wesfor = {'ACH-000004',
 'ACH-000118',
 'ACH-000246',
 'ACH-000258',
 'ACH-000272',
 'ACH-000304',
 'ACH-000356',
 'ACH-000444',
 'ACH-000452',
 'ACH-000488',
 'ACH-000658',
 'ACH-000738',
 'ACH-000768',
 'ACH-000837',
 'ACH-000848',
 'ACH-000865',
 'ACH-000960',
 'ACH-000961'}
historyofsnparray = {"ACH-000004","ACH-000014","ACH-000025","ACH-000028","ACH-000118","ACH-000128","ACH-000164","ACH-000167","ACH-000215","ACH-000237","ACH-000246","ACH-000258","ACH-000272","ACH-000304","ACH-000310","ACH-000356","ACH-000436","ACH-000444","ACH-000452","ACH-000454","ACH-000458","ACH-000488","ACH-000550","ACH-000591","ACH-000593","ACH-000600","ACH-000632","ACH-000634","ACH-000658","ACH-000673","ACH-000695","ACH-000738","ACH-000739","ACH-000768","ACH-000774","ACH-000836","ACH-000837","ACH-000848","ACH-000854","ACH-000865","ACH-000887","ACH-000901","ACH-000928","ACH-000941","ACH-000960","ACH-000961","ACH-001061","ACH-001239"}
stillsnpfor = historyofsnparray - wesfor
stillsnpfor

In [ ]:
stillsnpfor = stillsnpfor - set(df[df.datatype=='wes'].arxspan_id)
stillsnpfor

In [ ]:
set([i.split('_')[0] for i in cnsam.index]) & stillsnpfor

In [ ]:
h.listToFile(stillsnpfor,'droppedbcnoWES.txt')

# rescuinig ACH-003000

In [ ]:
for k, val in df.loc[df[df.ccle_name.isna()].index].iterrows():
    for v in df[df.arxspan_id==val.arxspan_id].ccle_name:
        if str(v) !='nan':
            df.loc[k,'ccle_name'] = v
            break

In [ ]:
df.loc[df[df.arxspan_id.isna()].index,'arxspan_id'] = ["ACH-001421","ACH-001533","ACH-001574","ACH-001678","ACH-001711","ACH-001847","ACH-002021","ACH-002048","ACH-002065","ACH-003000",'ACH-001957','ACH-000005']

#ACH-002446-311cas9

In [ ]:
df[df.arxspan_id=='ACH-003000']

In [ ]:
h.listToFile(['CCLFUPGI0005T-311cas9'],'tocreate_arxspan.txt')

# add other data type to csv

using the Terra workspacce CCLE2 and master depmap CV : pakita master  

using the columns : 
- Stripped Cell Line Name	
- Aliases	
- Gender	
- notes	
- Primary Site	
- Primary Disease	
- Subtype	
- Comments	
- Disease	
- Disease Subtype	
- Disease Sub-subtype
- Tumor Type for Dashboard

use snp pairs to: 
- merge some CL
- create parents of

In [ ]:
## QC file, sra, 

In [ ]:
masteromics.loc[4543,'DepMap ID']='ACH-003000'
masteromics.loc[4542,'DepMap ID']='ACH-003000'

In [ ]:
for val in set(df.arxspan_id):
    df.loc[df[df.arxspan_id==val].index,'patient_id'] = 'PT-'+h.randomString(stringLength=8, stype='all', withdigits=True)

In [ ]:
master = masterdepmap.find('Master_Paquita').to_frame(header=2)

In [ ]:
for k, val in master.iterrows():
    index =df[df.arxspan_id==val['DepMap_ID']].index
    df.loc[index,'stripped_cell_line_name'] = val["Stripped Cell Line Name"]
    df.loc[index,'gender'] = val["Gender"]
    df.loc[index,'primary_site'] = val["Primary Site"]
    df.loc[index,'primary_disease'] = val["Primary Disease"]
    df.loc[index,'subtype'] = val["Disease Subtype"]
    df.loc[index,'origin'] = val["Disease"]
    df.loc[index,'comments'] = val["Comments"]
    df.loc[index,'notes'] = val["Gender"]
    df.loc[index,'subsubtype'] = val["Disease Sub-subtype"]

In [ ]:
df.stripped_cell_line_name = [i.replace('-','').replace('_','') if type(i) is str else i for i in df.stripped_cell_line_name]

### Merging similar lines and adding parent relationships

In [ ]:
samepatient=[('HEL','HEL9217'),
('LS513','HCC1588'),
('C2BBE1','CACO2'),
('253J','253JBV'),
('A101D', 'HS294T'),
('HS294T', 'A101D'),
('SKBR3','AU565'),
('T24', 'ACCS'),
('MCF7', 'KPL1'),
('NCIH1693', 'NCIH1819'),
('PATU8988S', 'PATU8988T'),
('KPL1', 'MCF7'),
('U343', 'GOS3'),
('U118MG', 'U138MG' ),
('1321N1', 'U138MG'),
('TOV112D', 'HS571T'),
('PANC1005', 'PL45'),
('KP3L', 'KP3'),
('M059J', 'M059K'),
('SW579', 'CGTHW1'),
('KE97', 'KMS18'),
('PK1', 'KLM1'),
('MONOMAC1', 'MONOMAC6'),
('R256', 'R262'),
('PLCPRF5', 'ALEXANDER'),
('ISHIKAWA', 'HEK'),
('KARPAS422', 'OCILY10'),
('PSN1', 'GRM'),
('HLF', 'HLE'),
('NICH322', 'NICH3255'),
('FTC133', 'FTC238'),
('KMS18', 'KE97'),
('LN464', 'ACN'),
('HMC18', 'HLC1'),
('HEPG2', 'C3A'),
('AU565', 'SKBR3'),
('WM115', 'WM2664'),
('IGR39', 'IGR37SW620'),
('SW480', 'IGR37SW620'),
('COLO205', 'COLO201'),
('COLO206', 'COLO201'),
('COLO800', 'COLO794'),
('SF767', 'ME180'),
('WM2664', 'WM115'),
('PEO1', 'PEO4'),
('PEO6', 'PEO4'),
('GP5D', 'GP2D'),
('UPCISCC152', 'UPCISCC90'),
('MDAMB435S', 'M14'),
('MDAMB435', 'M14'),
('MEC1', 'MEC2'),
('SKMES1', 'RERFLCMA'),
('PANC1','KP1N'),
('IMR32','IMR5'),
('A545','NCIH1993'),
('NCIH2073','A545'),
('COLO775','RPMI8226'),
('COLO677','RPMI8226'),
('CHL1','COLO699'),
('HMCB','COLO699'),
('DLD1','HRT18'),
('HCT15','DLD1'),
('D458','D425'),
('CCLFPEDS0001T_KIDNEY','PEDS005TSUSP')]

sameline=[('ZR751','YMB1'),
('PC14','PC9'),
('A375C5', 'A373C6'),
('COLO206', 'COLO206F'),
('RH30','SJRH30')]

parentsof=[('ONCODG1', 'NIHOVCAR3'),
('MCIXC', 'SKNMC'),
('YMB1E', 'ZR751'),
('HEYA8', 'HEY'),
('A373C6', 'A375'),
('NC37', 'RAJI'),
('A2780ADR', 'A2780'),
('A2780cis', 'A2780'),
('MKN28', 'MKN74'),
('RT11284', 'RT112'),
('LNZTA3WT4', 'LNZ308'),
('LNZTA3WT11', 'LNZ308'),
('143B', 'HOS'),
('MDAMB435S', 'MDAMB435'),
('CORL23CPR', 'CORL23'),
('BE2M17', 'SKNBE2'),
('TUR', 'U937'),
('HEC1B', 'HEC1'),
('HEC1A', 'HEC1'),
('KP1NL', 'KP1N'),
('HAP1MLH1KO', 'HAP1')]

In [ ]:
for i,j in samepatient:
    if j in df.stripped_cell_line_name.tolist():
        df.loc[df[df.stripped_cell_line_name==i].index,'patient_id']=df[df.stripped_cell_line_name==j].patient_id[0]
        df.loc[df[df.stripped_cell_line_name==i].index,'gender']=df[df.stripped_cell_line_name==j].gender[0]
    else:
        print(i,j)

In [ ]:
for i,j in parentsof:
    if j in df.stripped_cell_line_name.tolist():
        df.loc[df[df.stripped_cell_line_name==i].index,'patient_id']=df[df.stripped_cell_line_name==j].patient_id[0]
        df.loc[df[df.stripped_cell_line_name==i].index,'gender']=df[df.stripped_cell_line_name==j].gender[0]
        df.loc[df[df.stripped_cell_line_name==i].index,'parent_cell_line'] = df[df.stripped_cell_line_name==j].arxspan_id[0]
    else:
        print(i,j)

In [ ]:
df.

In [ ]:
dfe = df.reset_index(drop=False).set_index('cds_id').drop(columns=['aliases','disease_subsubtype','disease_subtype','disease','tumor_type_for_dashboard']).rename(columns={'cds_idsm_id':'sm_id'})[['bam_public_sra_path','internal_bam_filepath', 'internal_bai_filepath', 'parent_cell_line', 'stripped_cell_line_name', 'gender', 'notes', 'primary_site', 'primary_disease', 'subtype', 'comments', 'arxspan_id', 'ccle_name', 'version', 'sm_id', 'source', 'datatype', 'sequencing_type', 'sequencing_date', 'baits', 'origin', 'subsubtype', 'patient_id', 'release_date', 'bam_qc', 'processing_qc', 'crc32c_hash', '18q1', '18q2', '18q3', '18q4', '19q1', '19q2', '19q3', '19q4', '20q1', '20q2']]

In [ ]:
dfe = dfe.drop(columns='notes')

In [ ]:
def inttodate(i, lim=1965):
    a = int(i//365)
    if a > lim:
        a = str(a)
        r = i%365
        m = str(int(r//30)) if int(r)>0 else str(1)
        r = r%30
        d = str(int(r)) if int(r)>0 else str(1)
    else:
        return 'U'
    return d+'/'+m+'/'+a  
dfe['sequencing_date'] = [inttodate(i) for i in dfe['sequencing_date']]

In [ ]:
import math

In [ ]:
i

In [ ]:
i%365

In [ ]:
dfe["condition"] = None
dfe["mediatype"] = None

In [ ]:
dfe

In [ ]:
dfe.to_csv('ccle_samples.csv')

In [ ]:
df =pd.read_csv('../data/ccle_samples - ccle_samples.csv').set_index('cds_id')

In [ ]:
clotoarx = pd.read_csv('../data/arxspan-cell-line-export_v390-ach.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
clotoarx

In [ ]:
for i,val in clotoarx.iterrows():
    loc = df[df.arxspan_id==val.arxspan_id].index
    df.loc[loc,'age'] = val['CCLF Age'] 
    df.loc[loc,'cellosaurus_id'] = val['RRID']
    for l in loc:
        if df.loc[l,'stripped_cell_line_name'] is np.nan:
            df.loc[l,'stripped_cell_line_name'] = val['Stripped Cell Line Name']

In [ ]:
df[df.stripped_cell_line_name.isna()]

In [ ]:
## check what is not in masteromics
set(df.arxspan_id) - set(masteromics['DepMap ID'])

In [ ]:
for val in df.cellosaurus_id:
    data= set(df[df.cellosaurus_id==val].arxspan_id)
    if len(data) > 1:
        print(data)   

In [ ]:
set(df[df.cellosaurus_id.isna()].stripped_cell_line_name)

In [ ]:
set(df[df.age.isna()].stripped_cell_line_name)

In [ ]:
set(df[df.gender.isna()].stripped_cell_line_name)

## get back the cellosaurus_id

In [ ]:
df.to_csv('../data/fullccle.csv')

In [ ]:
df = pd.read_csv('../data/fullccle.csv',index_col=0)
df

## get back the true age (or if ped or not)

In [ ]:
# we have pediatric != age

## get back from cellosaurus if same Cell Line

In [ ]:
# some cell lines have diffferent arxpsan id and or stripped cell line name but are the same cell line

In [ ]:
## reupdate the versions and make sure that annotations are the same (patient id, stripped name, gender, age, primary, disease, origin )

## get back from celosaurus if same patient

In [ ]:
# use known pairs file:
kp = pd.read_csv('data/Known Pairs - Main.csv')

In [ ]:
df[df.stripped_cell_line_name=='CMK86']

In [ ]:
import ipdb

In [ ]:
used= set()
for i,val in kp.iterrows():
    if val['Known Pairs'] is not np.nan:
        p = df[df.stripped_cell_line_name==val['Stripped Cell Line Name']].patient_id
        if len(p)==0:
            # we don't have this cell line
            print("we don't have this cl: "+ val['Stripped Cell Line Name'])
            continue
        else:
            p = p.values[0]
        for v in val['Known Pairs'].split(';'):
            v = v.replace(' ','').replace('*','')
            # if we don't have this cell line
            if len(v)<2 or v not in df.stripped_cell_line_name:
                continue
            if v in used:
                if val['Stripped Cell Line Name'] not in used:
                    #print(i, v,val['Stripped Cell Line Name'])
                    loc = df[df.stripped_cell_line_name==val['Stripped Cell Line Name']].index
                    p2=df[df.stripped_cell_line_name==v].patient_id
                    if len(p2)==0:
                        print("was empty")
                        continue
                    else:
                        p2 = p2.values[0]
                    df.loc[loc,'patient_id']= p2
                    continue
                else:
                    continue
            used.add(v)
            loc = df[df.stripped_cell_line_name==v].index
            df.loc[loc,'patient_id']=p
        used.add(val['Stripped Cell Line Name'])

In [ ]:
set(df[df.patient_id.isna()].stripped_cell_line_name) &set(kp['Stripped Cell Line Name'])

In [ ]:
df[df.stripped_cell_line_name.isin(["RPE1SS77","RPE1", "RPE1SS48", "RPE1SS6"])].patient_id

In [ ]:
kp.loc[10:]

In [ ]:
df.to_csv('../data/fullccle_matchedpatients.csv')

In [ ]:
# some cell lines have diffferent patient id but are actually from the same patient (~50)

In [ ]:
# and make sure that annotations are the same (patient id, gender, age, primary, disease..)

# Update Terra TSVs

In [ ]:
dfe[dfe.arxspan_id=='ACH-001092'].internal_bam_filepath

In [ ]:
set(dfe[dfe.gender.isna()].arxspan_id)

In [ ]:
dfe[dfe.datatype=='rna'][dfe.arxspan_id.isin(rnasam.participant)][dfe.version==1]

In [ ]:
df = pd.read_csv('../data/fullccle_matchedpatients.csv')

In [ ]:
rnawm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq')
rnasam = rnawm.get_samples()

cnwm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_WES_CN_hg38')
cnsam = cnwm.get_samples()

mutwm= dm.WorkspaceManager('broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline')
mutsam = mutwm.get_samples()

## get sm ids for all and change the ach ID for sm ID

In [ ]:
notinmyset = set([str(i).replace('/ccle_bams','/cclebams') for i in mutsam.WES_bam ]) -set(df.internal_bam_filepath)

In [ ]:
a = list(notinmyset)
a.remove('nan')

In [ ]:
gcp.lsFiles(a)

In [ ]:
newmutsam = pd.DataFrame(columns=mutsam.columns)

In [ ]:
df.columns.tolist()

In [ ]:
val.internal_bam_filepath.replace('cclebams','ccle_bams')

In [ ]:
mutsam.WES_bam

In [ ]:
baits = []
for i, val in df.iterrows():
    if val.datatype=='wes':
        if val.internal_bam_filepath.replace('cclebams','ccle_bams') in mutsam.WES_bam.tolist():
            print('i')
            baits.append( mutsam[ mutsam.WES_bam == val.internal_bam_filepath.replace('cclebams', 'ccle_bams')].target_type[0])
        else:
            baits.append('unknown')
    elif val.datatype=='rna':
        baits.append('polyA')
    else:
        baits.append(None)

In [ ]:
set(baits)

In [ ]:
baits.count('unknown')

In [ ]:
baits = [i if i not in ['NA', None] else np.nan for i in baits]

In [ ]:
df.baits=baits

In [ ]:
for i, v in df[df.baits=='unknown'].iterrows():
    if v.sequencing_date=='U':
        continue
    if int(v.sequencing_date.split('/')[-1]) <2016:
        df.loc[i,'baits']='AGILENT'
    elif int(v.sequencing_date.split('/')[-1]) >2017:
        df.loc[i,'baits']=='ICE'
    else:
        print(v)

In [ ]:
df.loc[5132,'baits'] = 'ICE'

In [ ]:
df.loc[df[df.baits=='unknown'].index,'baits']='ICE'

In [ ]:
df.sequencing_date = [i.replace('/0/','/1/') for i in df.sequencing_date]

In [ ]:
#because sanger is agilent and it is all sanger
df.loc[df[df.baits=='unknown'][df.sequencing_date=='U'].index,'baits']='AGILENT'

In [ ]:
df[df.datatype=='wes'][df.baits.isna()]

In [ ]:
df[df.stripped_cell_line_name=='HA7EBV']

In [ ]:
df[df.internal_bai_filepath.str.contains('NORMAL')]

In [ ]:
df.to_csv('../data/fullccle_matchedpatients.csv')

In [ ]:
df = pd.read_csv('../data/fullccle_matchedpatients.csv').set_index('cds_id', drop=True)

In [ ]:
rnas = df[df.datatype=='rna'].drop(columns=['bam_qc', 'processing_qc', 'crc32c_hash', '18q1','18q2', '18q3', '18q4', '19q1', '19q2', '19q3', '19q4', '20q1', '20q2','cellosaurus_id','ccle_name','comments']).rename(columns={'cds_id':'sample_id','patient_id':'participant_id'})

In [ ]:
rnas.index.name='sample_id'

In [ ]:
rnawm.disable_hound().upload_samples(rnas)

## make normal pairs and upload to Terra

In [ ]:
df[df.cellosaurus_id.isna() | df.gender.isna() | df.age.isna()].sort_values(by='stripped_cell_line_name').to_csv('../tofind.csv')

In [ ]:
for i in set(df[df.gender.isna()].patient_id):
    res = df[(df.patient_id==i) & (~df.gender.isna())]
    if len(res) > 0:
        print(res.gender[0])
        df.loc[df[df.patient_id==i].index,'gender']=res.gender[0]
for i in set(df[df.age.isna()].patient_id):
    res = df[(df.patient_id==i) & (~df.age.isna())]
    if len(res) > 0:
        print(res.age[0])
        df.loc[df[df.patient_id==i].index,'age']=res.age[0]
for i in set(df[df.primary_disease.isna()].patient_id):
    res = df[(df.patient_id==i) & (~df.primary_disease.isna())]
    if len(res) > 0:
        if res.primary_disease[0]!='normal':
            print(res.primary_disease[0])
            df.loc[df[df.patient_id==i].index,'primary_disease']=res.primary_disease[0]
for i in set(df[df.origin.isna()].patient_id):
    res = df[(df.patient_id==i) & (~df.origin.isna())]
    if len(res) > 0:
        print(res.origin[0])
        df.loc[df[df.patient_id==i].index,'origin']=res.origin[0]
for i in set(df[df.subtype.isna()].patient_id):
    res = df[(df.patient_id==i) & (~df.subtype.isna())]
    if len(res) > 0:
        if res.subtype[0]!='normal':
            print(res.subtype[0])
            df.loc[df[df.patient_id==i].index,'subtype']=res.subtype[0]

In [ ]:
a = h.fileToList('data/cellosaurus.txt')

In [ ]:
AG= {}
HI = {}
SX = {}
CVCL = {}
NAME = {}
ID=''
for val in a[10:]:
    if val[:2]=='AC':
        ID = val.split('   ')[1]
        CVCL[ID]=name
        NAME[name]=ID
    if val=='//':
        ID= ''
    if val[:2]=='AG' and ID:
        ag=val.split('   ')[1]
        ag = '1' if ag=='Yearling' else ag
        ag = ag.split('Y')[0].split('>')[-1].split('-')[0]
        ag = 'U' if ag == 'Age unspecified' else ag
        ag = 'A' if ag ==  'Adult' else ag
        ag = '0' if ag[-1] in ['M','W','D'] else ag
        ag = 'F' if ag=='Fetus' else ag
        af = 'P' if ag=='Children' else ag
        ag = 'E' if ag in ['Embryo','Embryonic stage','Late embryonic stage','Larva'] else ag
        ag = 'B' if ag == 'Blastocyst stage' else ag
        AG[ID] = ag
    if val[:2]=='SX' and ID:
        sx=val.split('   ')[1]
        SX[ID] = 'U' if sx=="Sex unspecified" else sx
    if val[:2]=='HI' and ID:
        if ID in HI:
            HI[ID].append(val.split('   ')[1].split(' ! ')[0])
        else:
            HI[ID] = [val.split('   ')[1].split(' ! ')[0]]
    if val[:2]=='ID':
        name = val.split('   ')[1]

In [ ]:
AG

In [ ]:
for i,v in df[df.cellosaurus_id.isna()].iterrows():
    if v.stripped_cell_line_name in NAME:
        df.loc[i,'cellosaurus_id'] = NAME[v.stripped_cell_line_name]
    else:
        df.loc[i,'cellosaurus_id']='U'

In [ ]:
for i,v in df[df.age.isna()| (df.age=='U')].iterrows():
    if v.cellosaurus_id in AG:
        #print(AG[v.cellosaurus_id])
        df.loc[i,'age'] = AG[v.cellosaurus_id]
    else:
        df.loc[i,'age'] = 'U'

In [ ]:
df.gender

In [ ]:
for i,v in df[df.gender.isna()| (df.gender=='U')].iterrows():
    if v.cellosaurus_id in SX:
        print(SX[v.cellosaurus_id])
        df.loc[i,'gender'] = SX[v.cellosaurus_id]
    else:
        df.loc[i,'gender'] = 'U'

In [ ]:
for i in HI.values():
    if len(i) >2:
        print(i)

In [ ]:
cello = set(df.cellosaurus_id)
for v in cello:
    if v in HI:
        ind_father = df[df.cellosaurus_id.isin(HI[v])].patient_id.tolist()
        if len(ind_father)==0:
            continue
        ind_child= df[df.cellosaurus_id==v].patient_id[0]
        ind_father.append(ind_child)
        #print(ind_father)
        if len(set(ind_father))<2:
            continue
        print(ind_father)
        df.loc[df[df.patient_id.isin([ind_father,ind_child])].index,'patient_id']=ind_father[0]
                

In [ ]:
df[df.gender.isna()]

In [ ]:
df.to_csv('../data/fullccle_matchedpatients.csv')

In [ ]:
df = pd.read_csv('../data/fullccle_matchedpatients.csv').set_index('cds_id', drop=True)

In [ ]:
muts = df[df.datatype=='wes'].drop(columns=['bam_qc', 'processing_qc', 'crc32c_hash', '18q1','18q2', '18q3', '18q4', '19q1', '19q2', '19q3', '19q4', '20q1', '20q2','cellosaurus_id','ccle_name','comments'])

### add the WGS files to WES pipeline

In [ ]:
lineswithonlyWGS= {'ACH-000014',
'ACH-000164',
'ACH-000454',
'ACH-000550',
'ACH-000632',
'ACH-000739',
'ACH-000928',
'ACH-001239'}

In [ ]:
muts = muts.append(df[df.datatype=='wgs'][df.arxspan_id.isin(lineswithonlyWGS)].drop(columns=['bam_qc', 'processing_qc', 'crc32c_hash', '18q1','18q2', '18q3', '18q4', '19q1', '19q2', '19q3', '19q4', '20q1', '20q2','cellosaurus_id','ccle_name','comments']))

In [ ]:
muts.index.name='sample_id'

In [ ]:
muts = muts.rename(columns={'patient_id':'participant_id'})

In [ ]:
mutwm = mutwm.disable_hound()

In [ ]:
mutwm.upload_samples(muts)

In [ ]:
cnwm.disable_hound().upload_samples(muts)

In [ ]:
nan = pd.DataFrame(columns=muts.columns, index=['nan'],data=[[None]*len(muts.columns)])
nan.index.name='sample_id'
nan.participant_id = ['nan']

In [ ]:
cnwm.disable_hound().upload_samples(nan)

In [ ]:
mutwm.disable_hound().upload_samples(nan)

In [ ]:
pairs = {}
for i,val in muts.iterrows():
    pairs[str(i)+'_'+str(val.matched_normal)] = [str(i),str(val.matched_normal),val.participant_id]

In [ ]:
pairs = pd.DataFrame.from_dict(pairs, orient='index',columns=['case_sample','control_sample','participant'])

In [ ]:
pairs.index.name = 'pair_id'

In [ ]:
pairs

In [ ]:
mutwm.upload_entities('pairs',pairs)

In [ ]:
cnwm.disable_hound().upload_entities('pairs',pairs)

# solve some RNAseq files we are not taking from external workspaces